# Conformer Minimization

This example will show how to optimize a conformer of paracetamol.

Load in a paracetamol molecule, generate a conformer for it, and perturb the conformer to ensure it needs minimization.

In [1]:
import openff.toolkit
import openff.units
import torch

molecule = openff.toolkit.Molecule.from_smiles("CC(=O)NC1=CC=C(C=C1)O")
molecule.generate_conformers(n_conformers=1)

conformer = torch.tensor(molecule.conformers[0].m_as(openff.units.unit.angstrom)) * 1.10
conformer.requires_grad = True

We specify that the gradient of the conformer is required so that we can optimize it using PyTorch.

Parameterize the molecule using OpenFF Interchange and convert it into a PyTorch tensor representation.

In [2]:
import openff.interchange

interchange = openff.interchange.Interchange.from_smirnoff(
    openff.toolkit.ForceField("openff-2.1.0.offxml"),
    molecule.to_topology(),
)

import smirnoffee.ff

force_field, [topology] = smirnoffee.ff.convert_interchange(interchange)

We can minimize the conformer using any of PyTorch's optimizers. 

In [3]:
import smirnoffee.potentials

optimizer = torch.optim.Adam([conformer], lr=0.02)

for epoch in range(75):
    energy = smirnoffee.potentials.compute_energy(
        topology.parameters, conformer, force_field
    )
    energy.backward()

    optimizer.step()
    optimizer.zero_grad()

    if epoch % 5 == 0 or epoch == 74:
        print(f"Epoch {epoch}: E={energy.item()} kJ / mol")

Epoch 0: E=427.2271423339844 kJ / mol
Epoch 5: E=29.65744972229004 kJ / mol
Epoch 10: E=-76.69727325439453 kJ / mol
Epoch 15: E=-92.81060791015625 kJ / mol
Epoch 20: E=-127.06442260742188 kJ / mol
Epoch 25: E=-154.0148162841797 kJ / mol
Epoch 30: E=-161.15867614746094 kJ / mol
Epoch 35: E=-169.47312927246094 kJ / mol
Epoch 40: E=-176.0826416015625 kJ / mol
Epoch 45: E=-176.5312957763672 kJ / mol
Epoch 50: E=-177.3107147216797 kJ / mol
Epoch 55: E=-178.5596466064453 kJ / mol
Epoch 60: E=-179.07481384277344 kJ / mol
Epoch 65: E=-179.67147827148438 kJ / mol
Epoch 70: E=-180.067626953125 kJ / mol
Epoch 74: E=-180.26405334472656 kJ / mol


We can then re-store the optimized conformer back into the molecule. Here we add the conformer to the molecule's conformer list, but we could also replace the original conformer.

In [4]:
molecule.add_conformer(conformer.detach().numpy() * openff.units.unit.angstrom)
molecule.visualize(backend="nglview")

NGLWidget(max_frame=1)